Preprocessing Steps:

1. Splitted the dataframe into 3 dataframes, textual, categorical and date
2. Converted the data of categorical dataframe from boolean to 0/1 
3. Reduced the dimensionality of the categorical dataframe using truncated SVD  
4. Created a recency dataframe using date dataframe comparing the dates to 11/01/2025
5. Lemmatized and Tokenized the data of the textual dataframes into two subframes and recombined them into a single dataframe
6. Vectorized the data using TFDIF and converted sparse matrix into a denser matrix for recombination
7. Recombined the three dataframes into one


In [40]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from sklearn.decomposition import TruncatedSVD
from datetime import datetime

In [2]:
dataset1=pd.read_csv(r"D:\hackathons\CaseVault\recommendation-engine\Data\preprocessed\encoded.csv")

In [3]:
dataset1

,Unnamed: 0,title,judgment_date,facts,"applicants_[""D'ASTA""]","applicants_[""D'OTTAVI""]","applicants_[""JEHOVAH'S WITNESSES OF MOSCOW"", 'OTHERS']","applicants_[""KEVIN O'DOWD""]","applicants_[""L'TRADING"", 'INVEST LINES KFT']","applicants_[""L'ÉRABLIÈRE A.S.B.L.""]",...,"gold_rationales_[5, 8, 11, 14, 15, 20, 22, 23, 27]","gold_rationales_[6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 20, 21, 29]","gold_rationales_[6, 7, 8, 9, 10, 11, 18]","gold_rationales_[6, 7, 8, 9, 10, 15, 19, 20, 21, 22]","gold_rationales_[6, 8, 11, 12, 27, 28, 30, 31, 32]","gold_rationales_[7, 9, 10, 11, 12]","gold_rationales_[8, 10, 12, 13, 14, 15]","gold_rationales_[8, 9, 10, 11, 12, 13]","gold_rationales_[9, 11, 13, 15, 21, 22, 23]",gold_rationales_[]
0,0,case of kosiński v poland,2016-02-09,5 the applicant was born in 1983 and is detain...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1,case of mescereacov v the republic of moldova,2016-02-09,5 the applicant was born in 1982 and is curren...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,2,case of shlychkov v russia,2016-02-09,5 the applicant was born in 1955 and lives in ...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,3,case of dallas v the united kingdom,2016-02-11,6 the applicant was born in 1977 and lives in ...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,4,case of mitrova and savik v the former yugosla...,2016-02-11,6 the applicants were born in 1983 and 2007 re...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,10995,case of kirakosyan v armenia no 2,2016-02-04,5 the applicant was born in 1960 and lives in ...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10996,10996,case of meier v switzerland,2016-02-09,5 the applicant was born in 1946 and is curren...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10997,10997,case of nazyrova and others v russia,2016-02-09,5 the applicants are russian nationals who at ...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10998,10998,case of baranov and others v russia,2016-02-09,8 in all cases the applicants brought civil pr...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [4]:
df_textual = dataset1[['title',	'facts']]

In [5]:
df_date = dataset1['judgment_date']

In [6]:
df_categorical = dataset1.drop(['title',	'judgment_date',	'facts'], axis=1)

In [7]:
df_textual

,title,facts
0,case of kosiński v poland,5 the applicant was born in 1983 and is detain...
1,case of mescereacov v the republic of moldova,5 the applicant was born in 1982 and is curren...
2,case of shlychkov v russia,5 the applicant was born in 1955 and lives in ...
3,case of dallas v the united kingdom,6 the applicant was born in 1977 and lives in ...
4,case of mitrova and savik v the former yugosla...,6 the applicants were born in 1983 and 2007 re...
...,...,...
10995,case of kirakosyan v armenia no 2,5 the applicant was born in 1960 and lives in ...
10996,case of meier v switzerland,5 the applicant was born in 1946 and is curren...
10997,case of nazyrova and others v russia,5 the applicants are russian nationals who at ...
10998,case of baranov and others v russia,8 in all cases the applicants brought civil pr...


In [8]:
df_categorical

,Unnamed: 0,"applicants_[""D'ASTA""]","applicants_[""D'OTTAVI""]","applicants_[""JEHOVAH'S WITNESSES OF MOSCOW"", 'OTHERS']","applicants_[""KEVIN O'DOWD""]","applicants_[""L'TRADING"", 'INVEST LINES KFT']","applicants_[""L'ÉRABLIÈRE A.S.B.L.""]","applicants_[""MARTINS O'NEILL PEDROSA""]","applicants_[""MEMBERS OF THE GLDANI CONGREGATION OF JEHOVAH'S WITNESSES"", 'OTHERS']","applicants_[""NATIONAL FEDERATION OF SPORTSPERSONS' ASSOCIATIONS"", 'UNIONS (FNASS)', 'OTHERS']",...,"gold_rationales_[5, 8, 11, 14, 15, 20, 22, 23, 27]","gold_rationales_[6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 20, 21, 29]","gold_rationales_[6, 7, 8, 9, 10, 11, 18]","gold_rationales_[6, 7, 8, 9, 10, 15, 19, 20, 21, 22]","gold_rationales_[6, 8, 11, 12, 27, 28, 30, 31, 32]","gold_rationales_[7, 9, 10, 11, 12]","gold_rationales_[8, 10, 12, 13, 14, 15]","gold_rationales_[8, 9, 10, 11, 12, 13]","gold_rationales_[9, 11, 13, 15, 21, 22, 23]",gold_rationales_[]
0,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,2,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,3,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,4,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,10995,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10996,10996,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10997,10997,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
10998,10998,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [9]:
df_date

0        2016-02-09
1        2016-02-09
2        2016-02-09
3        2016-02-11
4        2016-02-11
            ...    
10995    2016-02-04
10996    2016-02-09
10997    2016-02-09
10998    2016-02-09
10999    2016-02-09
Name: judgment_date, Length: 11000, dtype: object

In [10]:
def convert_true_false_to_ones_zeros(df):
    df = df.replace({True: 1, False: 0})
    return df.astype(int) 

df_categorical = convert_true_false_to_ones_zeros(df_categorical)


C:\Users\Uday\AppData\Local\Temp\ipykernel_15756\1820417594.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [11]:
df_categorical

,Unnamed: 0,"applicants_[""D'ASTA""]","applicants_[""D'OTTAVI""]","applicants_[""JEHOVAH'S WITNESSES OF MOSCOW"", 'OTHERS']","applicants_[""KEVIN O'DOWD""]","applicants_[""L'TRADING"", 'INVEST LINES KFT']","applicants_[""L'ÉRABLIÈRE A.S.B.L.""]","applicants_[""MARTINS O'NEILL PEDROSA""]","applicants_[""MEMBERS OF THE GLDANI CONGREGATION OF JEHOVAH'S WITNESSES"", 'OTHERS']","applicants_[""NATIONAL FEDERATION OF SPORTSPERSONS' ASSOCIATIONS"", 'UNIONS (FNASS)', 'OTHERS']",...,"gold_rationales_[5, 8, 11, 14, 15, 20, 22, 23, 27]","gold_rationales_[6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 20, 21, 29]","gold_rationales_[6, 7, 8, 9, 10, 11, 18]","gold_rationales_[6, 7, 8, 9, 10, 15, 19, 20, 21, 22]","gold_rationales_[6, 8, 11, 12, 27, 28, 30, 31, 32]","gold_rationales_[7, 9, 10, 11, 12]","gold_rationales_[8, 10, 12, 13, 14, 15]","gold_rationales_[8, 9, 10, 11, 12, 13]","gold_rationales_[9, 11, 13, 15, 21, 22, 23]",gold_rationales_[]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,10995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10996,10996,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10997,10997,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10998,10998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
df_categorical=df_categorical.drop(['Unnamed: 0'], axis=1)

In [13]:
df_categorical

,"applicants_[""D'ASTA""]","applicants_[""D'OTTAVI""]","applicants_[""JEHOVAH'S WITNESSES OF MOSCOW"", 'OTHERS']","applicants_[""KEVIN O'DOWD""]","applicants_[""L'TRADING"", 'INVEST LINES KFT']","applicants_[""L'ÉRABLIÈRE A.S.B.L.""]","applicants_[""MARTINS O'NEILL PEDROSA""]","applicants_[""MEMBERS OF THE GLDANI CONGREGATION OF JEHOVAH'S WITNESSES"", 'OTHERS']","applicants_[""NATIONAL FEDERATION OF SPORTSPERSONS' ASSOCIATIONS"", 'UNIONS (FNASS)', 'OTHERS']","applicants_[""O'BRIEN""]",...,"gold_rationales_[5, 8, 11, 14, 15, 20, 22, 23, 27]","gold_rationales_[6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 20, 21, 29]","gold_rationales_[6, 7, 8, 9, 10, 11, 18]","gold_rationales_[6, 7, 8, 9, 10, 15, 19, 20, 21, 22]","gold_rationales_[6, 8, 11, 12, 27, 28, 30, 31, 32]","gold_rationales_[7, 9, 10, 11, 12]","gold_rationales_[8, 10, 12, 13, 14, 15]","gold_rationales_[8, 9, 10, 11, 12, 13]","gold_rationales_[9, 11, 13, 15, 21, 22, 23]",gold_rationales_[]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
df_categorical.isna().sum()

applicants_["D'ASTA"]                                     0
applicants_["D'OTTAVI"]                                   0
applicants_["JEHOVAH'S WITNESSES OF MOSCOW", 'OTHERS']    0
applicants_["KEVIN O'DOWD"]                               0
applicants_["L'TRADING", 'INVEST LINES KFT']              0
                                                         ..
gold_rationales_[7, 9, 10, 11, 12]                        0
gold_rationales_[8, 10, 12, 13, 14, 15]                   0
gold_rationales_[8, 9, 10, 11, 12, 13]                    0
gold_rationales_[9, 11, 13, 15, 21, 22, 23]               0
gold_rationales_[]                                        0
Length: 21475, dtype: int64

In [15]:
df_categorical=df_categorical.fillna(0)

In [17]:
df_categorical

,"applicants_[""D'ASTA""]","applicants_[""D'OTTAVI""]","applicants_[""JEHOVAH'S WITNESSES OF MOSCOW"", 'OTHERS']","applicants_[""KEVIN O'DOWD""]","applicants_[""L'TRADING"", 'INVEST LINES KFT']","applicants_[""L'ÉRABLIÈRE A.S.B.L.""]","applicants_[""MARTINS O'NEILL PEDROSA""]","applicants_[""MEMBERS OF THE GLDANI CONGREGATION OF JEHOVAH'S WITNESSES"", 'OTHERS']","applicants_[""NATIONAL FEDERATION OF SPORTSPERSONS' ASSOCIATIONS"", 'UNIONS (FNASS)', 'OTHERS']","applicants_[""O'BRIEN""]",...,"gold_rationales_[5, 8, 11, 14, 15, 20, 22, 23, 27]","gold_rationales_[6, 7, 8, 10, 11, 12, 13, 14, 15, 17, 20, 21, 29]","gold_rationales_[6, 7, 8, 9, 10, 11, 18]","gold_rationales_[6, 7, 8, 9, 10, 15, 19, 20, 21, 22]","gold_rationales_[6, 8, 11, 12, 27, 28, 30, 31, 32]","gold_rationales_[7, 9, 10, 11, 12]","gold_rationales_[8, 10, 12, 13, 14, 15]","gold_rationales_[8, 9, 10, 11, 12, 13]","gold_rationales_[9, 11, 13, 15, 21, 22, 23]",gold_rationales_[]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
10998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
svd = TruncatedSVD(n_components=100)  
reduced_data = svd.fit_transform(df_categorical)

print(reduced_data.shape)


(11000, 100)


In [19]:
svd_df = pd.DataFrame(reduced_data)
svd_df.columns = [f'component_{i+1}' for i in range(svd_df.shape[1])]

In [20]:
svd_df

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,component_91,component_92,component_93,component_94,component_95,component_96,component_97,component_98,component_99,component_100
0,1.632583,-0.377532,0.421133,-0.572584,-0.131109,-0.001156,-0.595789,0.847357,0.465964,-0.257247,...,-0.001779,0.006357,0.011822,-0.007331,-0.017590,0.023313,-0.029550,0.001054,-0.006715,-0.028952
1,0.789583,-0.498816,-0.599629,-0.231379,0.016958,-0.591930,0.450830,-0.418475,0.848181,-0.488616,...,0.004880,-0.007016,0.005503,0.011388,0.001201,-0.002093,0.004204,0.000572,0.006611,-0.001555
2,0.849633,-0.589849,-0.655112,0.592220,-0.308401,-0.145752,-0.058529,-0.074482,-0.161176,0.132598,...,0.007780,0.005000,-0.005540,-0.011605,-0.011025,0.002027,-0.012009,0.009971,-0.003530,0.004941
3,0.783255,-0.403721,-0.562864,-0.392344,0.119531,-0.196019,-0.411185,0.621779,0.221999,0.307248,...,-0.044657,-0.038610,-0.006091,0.020393,0.016883,-0.064534,-0.019034,-0.006502,-0.041654,-0.028236
4,1.219752,-0.394178,-0.025055,-0.374735,0.006981,-0.212935,-0.390794,0.635433,0.208640,0.287776,...,0.015312,-0.003581,0.003116,0.008815,0.002529,-0.002203,-0.010721,0.001583,-0.030343,-0.020531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,0.778033,-0.405470,-0.574456,-0.375656,0.141227,-0.197933,-0.382736,0.574690,0.182558,0.283897,...,-0.020452,0.043864,-0.015005,-0.139351,-0.078345,0.128094,-0.040722,-0.005150,0.009622,-0.197819
10996,0.768899,-0.398344,-0.559442,-0.361337,0.128174,-0.201218,-0.381377,0.576052,0.197655,0.280361,...,-0.130802,-0.126418,-0.076333,0.003741,0.021628,0.078996,0.028006,-0.041402,-0.030744,0.018475
10997,1.639517,-0.587959,0.457809,0.675293,-0.637100,-0.181253,-0.134810,0.019063,-0.071621,0.150423,...,0.011595,-0.003339,0.019687,-0.010980,-0.008243,0.005220,-0.005028,0.007687,-0.013218,0.007012
10998,0.976004,-0.870889,-0.440822,1.160238,0.708630,0.226552,0.098795,0.148528,-0.160238,-0.200619,...,0.006888,0.011085,-0.007065,-0.001713,-0.005117,0.008288,0.023131,0.014514,0.003879,-0.019869


In [21]:
df_date = pd.to_datetime(df_date)
today = datetime.today()
df_recency = (today - df_date).dt.days

In [22]:
df_recency

0        3259
1        3259
2        3259
3        3257
4        3257
         ... 
10995    3264
10996    3259
10997    3259
10998    3259
10999    3259
Name: judgment_date, Length: 11000, dtype: int64

In [23]:
df_textual

,title,facts
0,case of kosiński v poland,5 the applicant was born in 1983 and is detain...
1,case of mescereacov v the republic of moldova,5 the applicant was born in 1982 and is curren...
2,case of shlychkov v russia,5 the applicant was born in 1955 and lives in ...
3,case of dallas v the united kingdom,6 the applicant was born in 1977 and lives in ...
4,case of mitrova and savik v the former yugosla...,6 the applicants were born in 1983 and 2007 re...
...,...,...
10995,case of kirakosyan v armenia no 2,5 the applicant was born in 1960 and lives in ...
10996,case of meier v switzerland,5 the applicant was born in 1946 and is curren...
10997,case of nazyrova and others v russia,5 the applicants are russian nationals who at ...
10998,case of baranov and others v russia,8 in all cases the applicants brought civil pr...


In [24]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 
nltk.download('wordnet')
nltk.download('punkt') 
nltk.download('punkt_tab')  
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Lemmatize each word
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join tokens back into a single string
    return " ".join(lemmatized_tokens)

df_lemmatized = df_textual['facts'].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Uday\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Uday\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Uday\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [25]:
df_lemmatized

0        5 the applicant wa born in 1983 and is detaine...
1        5 the applicant wa born in 1982 and is current...
2        5 the applicant wa born in 1955 and life in na...
3        6 the applicant wa born in 1977 and life in lu...
4        6 the applicant were born in 1983 and 2007 res...
                               ...                        
10995    5 the applicant wa born in 1960 and life in th...
10996    5 the applicant wa born in 1946 and is current...
10997    5 the applicant are russian national who at th...
10998    8 in all case the applicant brought civil proc...
10999    4 the applicant wa born in 1971 and life in pé...
Name: facts, Length: 11000, dtype: object

In [26]:
df_lemmatized2 = df_textual['title'].apply(lemmatize_text)

In [27]:
df_lemmatized2

0                                case of kosiński v poland
1            case of mescereacov v the republic of moldova
2                               case of shlychkov v russia
3                      case of dallas v the united kingdom
4        case of mitrova and savik v the former yugosla...
                               ...                        
10995                    case of kirakosyan v armenia no 2
10996                          case of meier v switzerland
10997                 case of nazyrova and others v russia
10998                  case of baranov and others v russia
10999                             case of balogh v hungary
Name: title, Length: 11000, dtype: object

In [28]:
df_textual2= pd.concat([df_lemmatized,df_lemmatized2],axis=1)

In [29]:
df_textual2.shape

(11000, 2)

In [30]:
df_textual2

,facts,title
0,5 the applicant wa born in 1983 and is detaine...,case of kosiński v poland
1,5 the applicant wa born in 1982 and is current...,case of mescereacov v the republic of moldova
2,5 the applicant wa born in 1955 and life in na...,case of shlychkov v russia
3,6 the applicant wa born in 1977 and life in lu...,case of dallas v the united kingdom
4,6 the applicant were born in 1983 and 2007 res...,case of mitrova and savik v the former yugosla...
...,...,...
10995,5 the applicant wa born in 1960 and life in th...,case of kirakosyan v armenia no 2
10996,5 the applicant wa born in 1946 and is current...,case of meier v switzerland
10997,5 the applicant are russian national who at th...,case of nazyrova and others v russia
10998,8 in all case the applicant brought civil proc...,case of baranov and others v russia


In [32]:
df_textual2['combined_text'] = (df_textual2['facts'].fillna('').str.lower() + " " + df_textual2['title'].fillna('')).str.lower()


In [33]:
df_textual2

,facts,title,combined_text
0,5 the applicant wa born in 1983 and is detaine...,case of kosiński v poland,5 the applicant wa born in 1983 and is detaine...
1,5 the applicant wa born in 1982 and is current...,case of mescereacov v the republic of moldova,5 the applicant wa born in 1982 and is current...
2,5 the applicant wa born in 1955 and life in na...,case of shlychkov v russia,5 the applicant wa born in 1955 and life in na...
3,6 the applicant wa born in 1977 and life in lu...,case of dallas v the united kingdom,6 the applicant wa born in 1977 and life in lu...
4,6 the applicant were born in 1983 and 2007 res...,case of mitrova and savik v the former yugosla...,6 the applicant were born in 1983 and 2007 res...
...,...,...,...
10995,5 the applicant wa born in 1960 and life in th...,case of kirakosyan v armenia no 2,5 the applicant wa born in 1960 and life in th...
10996,5 the applicant wa born in 1946 and is current...,case of meier v switzerland,5 the applicant wa born in 1946 and is current...
10997,5 the applicant are russian national who at th...,case of nazyrova and others v russia,5 the applicant are russian national who at th...
10998,8 in all case the applicant brought civil proc...,case of baranov and others v russia,8 in all case the applicant brought civil proc...


In [35]:
df_textual2=df_textual2.drop(['title','facts'],axis=1)

In [36]:
df_textual2

,combined_text
0,5 the applicant wa born in 1983 and is detaine...
1,5 the applicant wa born in 1982 and is current...
2,5 the applicant wa born in 1955 and life in na...
3,6 the applicant wa born in 1977 and life in lu...
4,6 the applicant were born in 1983 and 2007 res...
...,...
10995,5 the applicant wa born in 1960 and life in th...
10996,5 the applicant wa born in 1946 and is current...
10997,5 the applicant are russian national who at th...
10998,8 in all case the applicant brought civil proc...


In [37]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)  
tfidf_matrix = tfidf.fit_transform(df_textual2['combined_text'])

In [39]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3064143 stored elements and shape (11000, 5000)>

In [42]:
tfidf_dense = tfidf_matrix.toarray()
tfidf_df = pd.DataFrame(tfidf_dense, columns=tfidf.get_feature_names_out())

In [45]:
svd_df.shape

(11000, 100)

In [46]:
df_recency.shape

(11000,)

In [47]:

tfidf_dense.shape

(11000, 5000)

In [50]:
tfidf_dense_df = pd.DataFrame(tfidf_matrix.toarray())


In [51]:
df = pd.concat([svd_df, df_recency, tfidf_dense_df], axis=1)


In [52]:
df

,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,component_10,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,1.632583,-0.377532,0.421133,-0.572584,-0.131109,-0.001156,-0.595789,0.847357,0.465964,-0.257247,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.789583,-0.498816,-0.599629,-0.231379,0.016958,-0.591930,0.450830,-0.418475,0.848181,-0.488616,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.849633,-0.589849,-0.655112,0.592220,-0.308401,-0.145752,-0.058529,-0.074482,-0.161176,0.132598,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.783255,-0.403721,-0.562864,-0.392344,0.119531,-0.196019,-0.411185,0.621779,0.221999,0.307248,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,1.219752,-0.394178,-0.025055,-0.374735,0.006981,-0.212935,-0.390794,0.635433,0.208640,0.287776,...,0.0,0.0,0.0,0.0,0.025314,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10995,0.778033,-0.405470,-0.574456,-0.375656,0.141227,-0.197933,-0.382736,0.574690,0.182558,0.283897,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
10996,0.768899,-0.398344,-0.559442,-0.361337,0.128174,-0.201218,-0.381377,0.576052,0.197655,0.280361,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
10997,1.639517,-0.587959,0.457809,0.675293,-0.637100,-0.181253,-0.134810,0.019063,-0.071621,0.150423,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
10998,0.976004,-0.870889,-0.440822,1.160238,0.708630,0.226552,0.098795,0.148528,-0.160238,-0.200619,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [53]:
df.to_csv(r"D:\hackathons\CaseVault\recommendation-engine\Data\preprocessed\preprocessed.csv")